## Example: Seinfeld Food Database


In [1]:
import ibis
import os
from IPython.display import display, HTML
from markdown import markdown as md
DATADIR = "/home/shared"
from cdsutils.mutils import *
from cdsutils.sqlite import *
from dminteract.creator.utils import *

In [2]:
!cp /home/shared/seinfeld_utf8.sqlite3 .

In [3]:
conn = ibis.sqlite.connect("./seinfeld_utf8.sqlite3")
conn.list_tables()

['episodes', 'food_types', 'foods', 'foods_episodes']

In [4]:
tft = conn.table("food_types")

In [5]:
print(ibis.sqlite.compile(tft))

SELECT t0.id, t0.name 
FROM base.food_types AS t0


## What do these tables look like?
### Look at the Schema

In [6]:
for t in conn.list_tables():
    tbl = conn.table(t)
    nrows = tbl.count().execute()
    display(HTML(md("### Table: %s\n\nnumber of rows=%3d\n####Schema"%(t, nrows))))
    display(view_dict(tbl.schema(), vertical=False))

id,season,name,int32,int32,string


id,name,int32,string


id,type_id,name,int32,int32,string


food_id,episode_id,int32,int32


### Look at the actual data

# Putting Data Back Together

Since this is a relational database, data has been split across tables so that the data is in __First Normal Form__. To make the data useful we have to put the data back together with __joins__.

## Example: _What Food is in Each Episode?_

### What tables do we need to put together?

- `foods_episodes`: lists `food_id` and `episode_id`

Let's try joining `food` with `food_id` so we can see the names of the food

#### Problem!

Before we start, we need to recognize that we've got a problem. The columns in the tables have overlapping names:

- Every table has a column named `id`
- `foods`, `episodes` and `food_types` all have columns named `name`

#### Create table objects for our two tables

In [7]:
tf = conn.table("foods")
tfe = conn.table("foods_episodes")

### Join the tables

To join the tables we use the `join` method for which we need to specify which columns in each table refer to each other. For example `food_id` in `foods_episodes` refers to `id` in `foods`.

Both tables have a column named `id` so we will either have to discard one or both of them or rename one or both of them. To start with let's just discard the `id` columns. We will do this by specifically listing what columns to keep. 

In [8]:
tnew = tfe.join(tf, tfe.food_id==tf.id).projection([tf.name, tfe.episode_id])
itview(tnew)

interactive(children=(IntSlider(value=0, description='start', max=502), Output()), _dom_classes=('widget-inter…

#### Only Half way there

To make this useful, we still need to replace episode_id with something more useful. We need to join with `episodes`. `episode_id` refers to `id` in `episodes`.

Now we have to face the overlapping names, namely `name`. We can use the `name()` method to rename columns.

In [9]:
tf = conn.table("foods")
tfe = conn.table("foods_episodes")
te = conn.table("episodes")


tnew = tfe.join(tf, tfe.food_id==tf.id).join(te, tfe.episode_id==te.id).projection([tf.name, te.name.name("episode_name"), tfe.episode_id, te.season])
itview(tnew)

interactive(children=(IntSlider(value=0, description='start', max=502), Output()), _dom_classes=('widget-inter…

In [10]:
tnew.episode_id.distinct().count().execute(), tnew.episode_name.distinct().count().execute()

(140, 140)

## A Note on Line breaks in Python

The line above where we define `tnew` was very long, hence unreadable, and poor style. So we want to break the line up. However, Python is sensitive to indentation, so you to have careful with how lines are broken up. Here are two different ways we can break the lines up.

In [11]:
tnew = tfe.join(
    tf, tfe.food_id==tf.id).join(
        te, tfe.episode_id==te.id).projection(
            [tf.name, te.name.name("episode_name"), 
             tfe.episode_id, te.season])


In [12]:
tnew = tfe\
.join(tf, tfe.food_id==tf.id)\
.join(te, tfe.episode_id==te.id)\
.projection([tf.name, te.name.name("episode_name"), 
             tfe.episode_id, te.season])

__I kept both `tfe.episode_id` and `te.id` to demonstrate that they really are the same__

#### What are we still missing?

- The type of food
- `join` with `food_types`
    - `foods.type_id` refers to `food_types.id`
    - rename `food_types.name` to `food_category`

In [13]:
tf = conn.table("foods")
tfe = conn.table("foods_episodes")
te = conn.table("episodes")
tft = conn.table("food_types")

tnew = tfe.join(tf, tfe.food_id==tf.id)\
.join(te, tfe.episode_id==te.id)\
.join(tft, tft.id==tf.type_id)\
.projection(
    [tf.name, tft.name.name("food_category"),
     te.name.name("episode_name"), 
     tfe.episode_id, te.season, ])
itview(tnew)

interactive(children=(IntSlider(value=0, description='start', max=502), Output()), _dom_classes=('widget-inter…

#### `sort`

The order of our results seem less than ideal. We can sort our results using the `sort` method. `name` also seems too generic, so let's rename it to `food_name`.

In [14]:
tfe = conn.table("foods_episodes")
tf = conn.table("foods")
te = conn.table("episodes")
tft = conn.table("food_types")
tnew = te.join(tfe, tfe.episode_id== te.id)\
.join(tf, tfe.food_id==tf.id)\
.join(tft, tf.type_id==tft.id)\
.projection(
    [tf.name.name("food_name"),
     tft.name.name("food_category"),
     te.name.name("episode_name"), 
     tfe.episode_id, 
     te.season]).sort_by("episode_id")

In [15]:
itview(tnew)

interactive(children=(IntSlider(value=0, description='start', max=502), Output()), _dom_classes=('widget-inter…

# Filtering Data (`filter`/ SQL WHERE)

Now that we know how to put things together, we need to look at how to keep the parts we are interested in and discarding the rest. In Ibis this is [`filtering`](https://docs.ibis-project.org/notebooks/tutorial/2-Basics-Aggregate-Filter-Limit.html#Filtering) (corresponding to [`WHERE`](https://bit.ly/2Nv927e) in SQL).

We execute a filter by passing a list of expressions that evaluate to True or False.

__*Very Important Note*__: In Python if I want to ask if two things are equal, the symbol is __`==`__. A single __`=`__ is an assignment statement, not a question about equality.

#### Example

Select all the rows that include `Junkfood`

In [16]:
itview(tnew.filter([tnew.food_category == "Junkfood"]))

interactive(children=(IntSlider(value=0, description='start', max=69), Output()), _dom_classes=('widget-intera…

In [17]:
itview(tnew.filter([tnew.food_category == "Junkfood",
                    tnew.season < 6]))

interactive(children=(IntSlider(value=0, description='start', max=30), Output()), _dom_classes=('widget-intera…

### More Filters

This dataset is relatively simple in that there are only two types of data:

- Integers (e.g. `episode_id`, `season`)
- Strings (e.g. `food_name`, `food_category`)

Filtering with integers is straightroward (e.g. ==, >, <)

Filtering with strings is more complex.

#### Filtering with Strings: Partial Matching

- __Strings are case sensitive:__ "Mint" is not equal to "mint"
    - __Tip__: Consider convert everything to uppercase or lowercase
- __Pattern Matching:__ A variety of options here
    - `contains()`
        - Does the string contain the 
    - `like`
        - This uses wildcard matches: 
            - **_** matches any one character
            - __%__ matches any one OR MORE character(s)
    - `rlike`
        - This uses regular expressions. Regular expressions are VERY powerful and VERY trick and so VERY frustrating
        - For those who are interested, I'll provide regular expression  materials later.

#### Example: Finding Minty Foods

In [18]:
tnew.filter([tnew.food_name.contains("mint")]).execute()

,food_name,food_category,episode_name,episode_id,season
0,Doublemint Gum,Junkfood,The Pie,78,5


In [19]:
tnew.filter([tnew.food_name.lower().contains("mint")]).execute()

,food_name,food_category,episode_name,episode_id,season
0,Mints,Junkfood,The Trip 1,40,4
1,Junior Mints,Junkfood,The Junior Mint,59,4
2,Doublemint Gum,Junkfood,The Pie,78,5
3,Chocolate Chip Mint,Junkfood,The Wait Out,132,7
4,Mints,Junkfood,The Cartoon,168,9


#### Example: Like

Find all foods that start with "P" in season 5.

In [20]:
tnew.filter([tnew.food_name.like("P%"),tnew.season==5]).execute()

,food_name,food_category,episode_name,episode_id,season
0,Papaya,Fruit,The Mango,64,5
1,Peaches,Fruit,The Mango,64,5
2,Plantains,Fruit,The Mango,64,5
3,Plumbs,Fruit,The Mango,64,5
4,Papaya,Fruit,The Glasses,66,5
5,Pizza,Rice/Pasta,The Sniffing Accountant,67,5
6,Pea Soup,Soup,The Lip Reader,69,5
7,Peas,Vegetables,The Lip Reader,69,5
8,Prune Juice,Drinks,The Cigar Store Indian,73,5
9,Pie,Bakery,The Stall,75,5


### Negating Filters

- You can negate a filter by prepending it with a negative sign (__-__)

In [21]:
tnew.filter([-tnew.food_name.like("P%"),tnew.season==5]).execute()

,food_name,food_category,episode_name,episode_id,season
0,Avacado,Fruit,The Mango,64,5
1,Cantaloupe,Fruit,The Mango,64,5
2,Mangos,Fruit,The Mango,64,5
3,Rissoto,Rice/Pasta,The Mango,64,5
4,Grilled Cheese,Sandwiches,The Mango,64,5
...,...,...,...,...,...
57,Bologna,Sandwiches,The Opposite,85,5
58,Chicken Salad on Rye,Sandwiches,The Opposite,85,5
59,Tuna on toast,Sandwiches,The Opposite,85,5
60,Salmon,Seafood,The Opposite,85,5


### Introducing `count`

Often times we just want to see how many rows there are that match a condition.

Ibis provides a `count` method that will count the number of resuslting rows.

So what if we want to count the number of foods with a capital `P`.

In [22]:
tnew.filter([tnew.food_name.like("%P%"),tnew.season==5]).count().execute()

29

### What if we want to see how many `p`s there are, upper or lower?

Remember we have the `upper` and `lower` methods to convert strings to uppercase or lowercase.

In [23]:
tnew.filter([tnew.food_name.upper().like("%P%"),tnew.season==5]).count().execute()

29

In [24]:
for w in create_question_bank("day1_bec.yaml", tag="ibis, food, case"):
    display(w)

TFWidget(children=(Label(value='Question'), HTML(value='<p>The best explanation of what just happened is that …

### `AND` and `OR`

Providing multiple filters acts like a logical `AND`:

"Show me all the rows where `food_name` has a `P` __AND__ the eposide is in season 6."

So if I provide two mutually exclusive filters, I will get no results.

"Show me all the rows where an episode is in season 5 __AND__ the episode is in season 6."

In [25]:
tnew.filter([tnew.season==5,tnew.season==6]).execute()

,food_name,food_category,episode_name,episode_id,season


### `OR` in Ibis

- An OR question is something like:
 - "Show me all the rows where there is a food name with a `P` and the episode is in season 5 __OR__ in season 6.

- I can create an `OR` with the symbol __`|`__.
    - There is a similar `AND` operator: `&`

__NOTE__: I had to wrap each question for the OR in parentheses so that the `|` operator didn't try to apply to the number 5 and `tnew.season`.



In [26]:
tnew.filter([tnew.food_name.like("%P%"),(tnew.season==5) | (tnew.season==6)]).execute()

,food_name,food_category,episode_name,episode_id,season
0,Cantaloupe,Fruit,The Mango,64,5
1,Papaya,Fruit,The Mango,64,5
2,Peaches,Fruit,The Mango,64,5
3,Plantains,Fruit,The Mango,64,5
4,Plumbs,Fruit,The Mango,64,5
5,Cantaloupe,Fruit,The Glasses,66,5
6,Papaya,Fruit,The Glasses,66,5
7,Rold Gold Pretzles,Junkfood,The Glasses,66,5
8,Ketchup,Condiments,The Sniffing Accountant,67,5
9,Pizza,Rice/Pasta,The Sniffing Accountant,67,5


If I pass a list of conditions to `like` this acts like an __OR__

In [27]:
tnew.filter([tnew.food_name.like(["%Pepsi%", "%Cola%"])]).execute()

,food_name,food_category,episode_name,episode_id,season
0,Chocolate Covered Cherries,Junkfood,The Stake Out,2,1
1,Pepsi,Drinks,The Truth,19,3
2,Godiva Chocolates,Junkfood,The Stall,75,5
3,Chocolate Bobka,Bakery,The Dinner Party,76,5
4,Pepsi,Drinks,The Dinner Party,76,5
5,Chocolate Cream Pie,Bakery,The Pie,78,5
6,"Chocolates, box of",Junkfood,The Wife,80,5
7,Chocolate Eclairs,Bakery,The Gymnast,91,6
8,Chocolate Chip Mint,Junkfood,The Wait Out,132,7
9,Coca Cola,Drinks,The Abstinence,142,8


In [28]:
for w in create_question_bank("day1_bec.yaml", tag="ibis,food,like"):
    display(w)


MCWidget(children=(Label(value='Select the Correct Answer'), HTML(value='<h4>Fixing the <code>like</code> quer…

## Getting Distinct/Unique Values

Remember that in relational databases, we are aiming for at least 1st normal form: one distince value in each cell. So in our results we often have multiple rows with the same episode or the same food. What if we want to count the number of episodes that mention pie? Then we can use the `distinct` method.

In [29]:
tnew.filter(tnew.food_name.like("%pie%")).episode_id.distinct().execute()

0     46
1     75
2     78
3    115
4    140
Name: episode_id, dtype: int32

In [30]:
tnew.filter([tnew.food_name.like("%P%"),(tnew.season==5) | (tnew.season==6)]).episode_id.distinct().count().execute()

22

## Missing Values

When we execute an Ibis query, it returns a Pandas dataframe. In Pandas `NaN` indicates a missing value. In this data set,the _Pilot_, the very first episode of the TV show is not in a season so the `season` value is `NaN`.

In [31]:
tnew.execute(limit=5)

,food_name,food_category,episode_name,episode_id,season
0,Salad Dressing,Condiments,Good News Bad News,0,NaN
1,Decaf Coffee,Drinks,Good News Bad News,0,NaN
2,Meat Loaf,Meat,Male Unbonding,1,1.0
3,Pizza,Rice/Pasta,Male Unbonding,1,1.0
4,Egg Salad,Sandwiches,Male Unbonding,1,1.0


### `notnull`

Ibis provides a `notnull` method that allows us to filter by out missing values.

In [32]:
tnew.filter([tnew.food_category=="Drinks", tnew.season.notnull()]).sort_by("food_name").execute()

,food_name,food_category,episode_name,episode_id,season
0,Bosco,Drinks,The Baby Shower,8,2
1,Club Soda,Drinks,The Jacket,9,2
2,Cranberry juice with two limes,Drinks,The Jacket,9,2
3,Scotch,Drinks,The Jacket,9,2
4,Coffe,Drinks,The Phone Message,11,2
...,...,...,...,...,...
71,Peach Schnapps,Drinks,The Betrayal,163,9
72,Cocoa,Drinks,The Bookstore,172,9
73,Dr. Pepper (diet),Drinks,The Puerto Rican Day,175,9
74,Royal Crown Cola,Drinks,The Puerto Rican Day,175,9


In [33]:
tnew.filter([tnew.season.notnull()]).sort_by("food_name").execute(limit=10)

,food_name,food_category,episode_name,episode_id,season
0,Egg Salad,Sandwiches,Male Unbonding,1,1
1,Meat Loaf,Meat,Male Unbonding,1,1
2,Pizza,Rice/Pasta,Male Unbonding,1,1
3,Turkey Roll,Sandwiches,Male Unbonding,1,1
4,Bouillabaisse,Soup,The Stake Out,2,1
5,Chocolate Covered Cherries,Junkfood,The Stake Out,2,1
6,BLT,Sandwiches,The Robbery,3,1
7,Brisket Sandwich,Sandwiches,The Robbery,3,1
8,Egg Salad,Sandwiches,The Stock Tip,4,1
9,Grape,Fruit,The Stock Tip,4,1


## Practice

In [34]:
for w in create_question_bank("day1_bec.yaml", tag="ibis,practice"):
    display(w)

TFWidget(children=(Label(value='Question'), HTML(value='<p>There are no duplicated episode names.</p>'), Radio…

MCWidget(children=(Label(value='Select the Correct Answer'), HTML(value='<p>What is the name of the episode th…

TFWidget(children=(Label(value='Question'), HTML(value='<p>Six seasons have episodes that mention some form of…

ATWidget(children=(Label(value='Select all True Answers'), HTML(value='<p>Which of the following episodes DOES…

TFWidget(children=(Label(value='Question'), HTML(value='<p>There are three episodes in season 4 that mention s…